In [20]:
%matplotlib inline
import pandas as pd

#Importo Decisions.csv
decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Da "Decisions.csv" seleziono solo le tuple riguardanti il primo round
decisions1 = decisions[decisions["Round"] == 9]

#Ho effettuato un controllo esterno per assicurarmi che tutte le tuple vengano prese in considerazione

#Estraggo "ID", "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions1 = decisions1.iloc[:, [0, 1, 3, 4]]

#Da "Decisions.csv" seleziono solo le tuple riguardanti il secondo round
decisions2 = decisions[decisions["Round"] == 10]

#Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions2 = decisions2.iloc[:, [1, 3, 4]]

#Importo TasksQualities per le task assegnate a ciascun WA
queue = pd.read_csv("../Dataset 2.0/TasksQualities.csv", sep = ";")

queue = queue.sort_values(by = "ID")

#Creo un DataFrame vuoto nel quale concatenerò il risultato di ciascuna iterazione che sto per eseguire
totale = pd.DataFrame()


for i in range(20):
    
    #Faccio coincidere in entrambe le tabelle gli wa scelti (in due round consecutivi)
    tmp1 = decisions1[decisions1["Worker Agent ID"] == i+1]
    tmp2 = decisions2[decisions2["Worker Agent ID"] == i+1]
    tmp1 = pd.merge(tmp1, tmp2, left_on = "Session ID", right_on = "Session ID")
    tmp1 = tmp1.rename(columns = {"Worker Agent ID_x": "Worker Agent ID", "Worker Agent Reputation_x": "Worker Agent Starting Reputation", "Worker Agent Reputation_y": "Worker Agent Observed Reputation"})
    tmp1 = pd.merge(queue, tmp1, left_on = "ID", right_on = "ID")
    totale = pd.concat([totale, tmp1])
    
totale = totale.sort_values(by = "ID")
totale = totale.drop("Worker Agent ID_y", 1)
totale = totale.drop("Session ID", 1)


x = totale.iloc[:, 0:93]
y = totale.iloc[:, 93]

/tmp/ipykernel_7852/3363804680.py:41: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  totale = totale.drop("Worker Agent ID_y", 1)
/tmp/ipykernel_7852/3363804680.py:42: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  totale = totale.drop("Session ID", 1)


In [21]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x, y)
reg.score(x, y)

0.9537148596502453

In [22]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#import numpy as np

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(x, y)
regr.score(x, y)

0.788342762977449